<h1 style='background-color: BLACK; padding: 10px; color: white'> Zoopla </h1>

<h1 style='background-color: BLACK; padding: 10px; color: white'> Sales </h1>


First things first, we will import the required libraries

In [1]:
import undetected_chromedriver as webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

Next, we will load in the dataset containing the list of postcode for this project

In [2]:
codes = pd.read_csv('London postcode districts.xlsx - PC DIST.csv')

Now, we will create a function called 'zoopla_sales' to scrap properties that are for sale on the Zoopla website

In [17]:
def zoopla_sales(postcodes, Trans_type, website, df):
    """
    This function scrapes property data from Rightmove for a given list of postcodes and transaction type,
    and returns the data as a pandas DataFrame.

    Arguments:
    postcodes -- a list of postcodes for which to scrape property data
    Trans_type -- the transaction type of the properties to be scraped ('sales' or 'rent')
    website -- the name of the website being scraped (in this case, 'Rightmove')
    df -- an empty pandas DataFrame to store the scraped data

    Returns:
    df1 -- a pandas DataFrame containing the scraped property data
    
    """    
    # Set the path to the ChromeDriver executable
    s = Service("C:\\Users\\user\\Downloads\\Set ups\\chromedriver_win32\\chromedriver.exe")
    
    # Launch the ChromeDriver with the specified service
    driver = webdriver.Chrome(service= s)
    
    # Navigate to the specified URL 
    driver.get('https://www.zoopla.co.uk/') 

    # Click to cancel the pop-up window and maximize the window
    #driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div[2]/button').click()
    time.sleep(2)
    driver.maximize_window()

    # Find the search bar  
    time.sleep(1.2)
    search = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/main/div[1]/div/div/div[1]/div[2]/div/div/div[2]/div/form/div/div[1]/div/div/div/div/div/div/input')
    search.send_keys(postcode)
    time.sleep(1.3)
    
    
    # Click the search button
    time.sleep(3)
    driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/main/div[1]/div/div/div[1]/div[2]/div/div/div[2]/div/form/div/div[2]/button').click()

    
    
    # Initialize empty lists for storing scraped data
    Trans_type = []
    address = []
    types = []
    bedrooms = []
    bathrooms = []
    prices = []
    desc = []
    date_added = []
    agent_list = []
    property_url = []
    website = []
    
    # Create an empty dataframe to store the scraped data
    df = pd.DataFrame()
    
     # Setting the page number to be 1 
    i = 1
    while True: 
        # Scrapping data for the required features in the first page
        time.sleep(2) 
        print("{} {} {} {}".format('scraping page', i,'from', postcode ))
        try:                                               
            address_list = driver.find_elements(By.XPATH, '//div/div/div/div/div[2]/div/a/div/div[3]/h3') 
        except:
            address_list = driver.find_elements(By.XPATH, '//div/div/div/div[2]/div[2]/div/a/div/div[3]/h3')
            
        try:                                             
            type_list = driver.find_elements(By.XPATH, "//div/div/div/div/div[2]/div/a/div/div[3]/h2")
            type_list = driver.find_elements(By.XPATH, "//div/div/div/div/div[2]/div/a/div/div[3]/h3")
        except:
            type_list = driver.find_elements(By.XPATH, "//div/div/div/div[2]/div[2]/div/a/div/div[3]/h2")
        
        try:                                               
            bedroom_list = driver.find_elements(By.XPATH, '//div/div/div/div/div[2]/div/a/div/div[2]/ul/li[1]/span[2]')
        except: 
            bedroom_list = driver.find_elements(By.XPATH, '//div/div/div/div[2]/div[2]/div/a/div/div[2]/ul/li[1]/span[2]')
            
        try:                                                
            bathroom_list = driver.find_elements(By.XPATH, '//div/div/div/div/div[2]/div/a/div/div[2]/ul/li[2]/span[2]')
        except: 
            bathroom_list = driver.find_elements(By.XPATH, '//div/div/div/div[2]/div[2]/div/a/div/div[2]/ul/li[2]/span[2]')
        
        try:                                            
            price_list =driver.find_elements(By.XPATH, '//div/div/div/div/div[2]/div/a/div/div[1]/div/p')
            price_list =driver.find_elements(By.XPATH, '//div/div/div/div/div[2]/div/a/div/div[1]/div/p[2]')
        except:
            price_list =driver.find_elements(By.XPATH, '//div/div/div/div/div[2]/div/a/div/div[1]/div/p')
            
        try:                                            
            desc_list = driver.find_elements(By.XPATH, '//div/div/div/div/div[2]/div/a/div/div[3]/p') 
        except:
            desc_list = driver.find_elements(By.XPATH, '//div/div/div/div[2]/div[2]/div/a/div/div[3]/p')
            
        try:                                                  
            date_added_list = driver.find_elements(By.XPATH, '//div/div/div/div/div[2]/div/a/div/ul/li') 
        except:
            date_added_list = driver.find_elements(By.XPATH, '//div/div/div/div/div[2]/div/a/div/ul[2]/li')
        try:
            agent_list_l = driver.find_elements(By.XPATH, '//div/div/div/div/div[3]/a/img')
            html1 = agent_list_list.get_attribute('outerHTML')
            pattern = r'alt="([^"]+)" loading='
            match = re.search(pattern, html1)
            agent_list_list = match.group(1)
        except:
            agent_list_list = price_list
        
        property_url_list = driver.current_url
        Trans_type_list = Trans_type
        website_list = website
        for address_item, type_item, bedroom_item, bathroom_item, price_item, desc_item, date_added_item, agent_list_item in zip(address_list, type_list, bedroom_list, bathroom_list, price_list, desc_list, date_added_list, agent_list_list):
            address.append(address_item.text)
            types.append(type_item.text)
            bedrooms.append(bedroom_item.text)
            bathrooms.append(bathroom_item.text)
            prices.append(price_item.text)
            desc.append(desc_item.text)
            date_added.append(date_added_item.text)
            agent_list.append(agent_list_item.text)
            property_url.append(property_url_list)
            Trans_type.append(Trans_type_list)
            website.append(website_list)


        time.sleep(1.3)
        # get the height of the page
        page_height = driver.execute_script("return document.body.scrollHeight;")

        # scroll to the middle of the page using JavaScript
        driver.execute_script(f"window.scrollTo(0, {page_height//1.37});")
        
        # Cancel the pop-up window
        
        try: 
            driver.find_element(By.XPATH, '/html/body/div[3]/div/div/main/div/div[5]/div[2]/main/div[1]/button').click()
        
        except:
            time.sleep(1.0)
            
                                                    
         # Find the next button  /html/body/div[3]/div/div/main/div/div[4]/div[2]/section/div[2]/div[3]/nav/div[3]/div/a
        try:
            time.sleep(2.2)                              
            #next_botton = driver.find_element(By.XPATH, '//div/div[4]/div[2]/section/div[2]/div[4]/nav/div[3]/div/a')
            next_botton = driver.find_element(By.CSS_SELECTOR, '#main-content > div > div._7t8z7g0 > div._7t8z7g1 > section > div._1g0c2na1 > div:nth-child(4) > nav > div._13wnc6k2 > div > a')
            
            # Find the last next button 
            #next_botton = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/main/div/div[4]/div[2]/section/div[2]/div[3]/nav/div[3]/div/a')
        
        except:
            time.sleep(2.1) 
            #next_botton = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/main/div/div[4]/div[2]/section/div[2]/div[3]/nav/div[3]/div/a')
            
            next_botton = driver.find_element(By.CSS_SELECTOR, '#main-content > div > div._7t8z7g0 > div._7t8z7g1 > section > div._1g0c2na1 > div:nth-child(3) > nav > div._13wnc6k2 > div > a')
        
        # Break if this is the last page 
        time.sleep(2.1)
        html = next_botton.get_attribute('outerHTML')
        if "#__next" not in html:
            print('------------------------------- SCRAPING COMPLETED FOR ' + postcode)
            break
            
        #Click the next button if it's not the last 
        time.sleep(8)
        next_botton.click()
        
        #Increment the page number
        i += 1
        time.sleep(2)
                
        
    # Create a dataframe to store data scrapped for each postcode    
    df1 = pd.DataFrame({'Location': postcode, 'Tansaction_Type': 'Sales','Property_Type':types, 'Address' :address, 'Bedrooms': bedrooms, 'Bathrooms':bathrooms, 'Price':prices, 'Description': desc, 'Listing_Date':date_added, 'Agent':agent_list, 'Listing_Source': 'Zoopla', 'listing_URL':property_url})
    df1.insert(0, 'Unique_Id', [f'{postcode}S{i+1:05d}ZP' for i in range(len(df1))])
    print('Total numbers of properties available in ' + postcode + ' is ' + str(df1.shape[0]))

     # Concat the dataframe obtain for all postcodes
    df = pd.concat([df, df1], ignore_index=True)
    
     # Return a dataframe
    return df1



In [18]:
# create an empty DataFrame outside the function
df = pd.DataFrame()

# loop through postcodes
for postcode in codes.loc[264:]['Postcode district']:
    # call the function and pass the empty DataFrame as an argument
    df1 = zoopla_sales(postcode, 'Sales', 'Zoopla', df)
    # append the df1 DataFrame to the empty DataFrame
    df = pd.concat([df, df1], ignore_index=True)
print('------------------------ WEB SCRAPING COMPLETED: OVERALL TOTAL NUMBER OF PROPERTIES SCRAPED IS ' + str(df.shape[0]))



scraping page 1 from WC2E
------------------------------- SCRAPING COMPLETED FOR WC2E
Total numbers of properties available in WC2E is 7
scraping page 1 from WC2H
------------------------------- SCRAPING COMPLETED FOR WC2H
Total numbers of properties available in WC2H is 11
scraping page 1 from WC2N
------------------------------- SCRAPING COMPLETED FOR WC2N
Total numbers of properties available in WC2N is 0
scraping page 1 from WC2R


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div/main/div/div[4]/div[2]/section/div[2]/div[4]/nav/div[3]/div/a"}
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00EA8893+48451]
	(No symbol) [0x00E3B8A1]
	(No symbol) [0x00D45058]
	(No symbol) [0x00D70467]
	(No symbol) [0x00D7069B]
	(No symbol) [0x00D9DD92]
	(No symbol) [0x00D8A304]
	(No symbol) [0x00D9C482]
	(No symbol) [0x00D8A0B6]
	(No symbol) [0x00D67E08]
	(No symbol) [0x00D68F2D]
	GetHandleVerifier [0x01108E3A+2540266]
	GetHandleVerifier [0x01148959+2801161]
	GetHandleVerifier [0x0114295C+2776588]
	GetHandleVerifier [0x00F32280+612144]
	(No symbol) [0x00E44F6C]
	(No symbol) [0x00E411D8]
	(No symbol) [0x00E412BB]
	(No symbol) [0x00E34857]
	BaseThreadInitThunk [0x75E20099+25]
	RtlGetAppContainerNamedObjectPath [0x77897B6E+286]
	RtlGetAppContainerNamedObjectPath [0x77897B3E+238]


In [ ]:
<img class="_12bxhf70" src="https://st.zoocdn.com/zoopla_static_agent_logo_(310780).
png" alt="Proctors - Bromley" loading="lazy">

<img class="_12bxhf70" src="https://st.zoocdn.com/zoopla_static_agent_logo_(311656).
png" alt="Coady Phillips" loading="lazy">



In [54]:
df.to_csv('zoopla_test1.csv')

In [52]:
data = codes.loc[266:]
data

,Postcode district,Local Areas,Borough
266,WC2N,Charing Cross,Westminster
267,WC2R,Somerset House,Westminster


In [16]:
s = Service("C:\\Users\\user\\Downloads\\Set ups\\chromedriver_win32\\chromedriver.exe")

# Launch the ChromeDriver with the specified service
driver = webdriver.Chrome(service= s)

# Navigate to the specified URL 
driver.get('https://www.zoopla.co.uk/') 

# Click to cancel the pop-up window and maximize the window
#driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div/div[2]/button').click()
time.sleep(2)
driver.maximize_window()

# Find the search bar  
time.sleep(1.2)
search = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/main/div[1]/div/div/div[1]/div[2]/div/div/div[2]/div/form/div/div[1]/div/div/div/div/div/div/input')
search.send_keys('BR1')
time.sleep(1.3)


# Click the search button
time.sleep(3)
driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/main/div[1]/div/div/div[1]/div[2]/div/div/div[2]/div/form/div/div[2]/button').click()
try:                                          
    agent_list_l = driver.find_element(By.XPATH, '//div/div/div/div/div[3]/a/img')
except: 
     agent_list_l = driver.find_element(By.XPATH, '//div[1]/div/div/div/div[3]/a/img')
    
html1 = agent_list_list.get_attribute('outerHTML')
pattern = r'alt="([^"]+)" loading='
match = re.search(pattern, html1)
agent_list_list = match.group(1)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[1]/div/div/div/div[3]/a/img"}
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00EA8893+48451]
	(No symbol) [0x00E3B8A1]
	(No symbol) [0x00D45058]
	(No symbol) [0x00D70467]
	(No symbol) [0x00D7069B]
	(No symbol) [0x00D9DD92]
	(No symbol) [0x00D8A304]
	(No symbol) [0x00D9C482]
	(No symbol) [0x00D8A0B6]
	(No symbol) [0x00D67E08]
	(No symbol) [0x00D68F2D]
	GetHandleVerifier [0x01108E3A+2540266]
	GetHandleVerifier [0x01148959+2801161]
	GetHandleVerifier [0x0114295C+2776588]
	GetHandleVerifier [0x00F32280+612144]
	(No symbol) [0x00E44F6C]
	(No symbol) [0x00E411D8]
	(No symbol) [0x00E412BB]
	(No symbol) [0x00E34857]
	BaseThreadInitThunk [0x75E20099+25]
	RtlGetAppContainerNamedObjectPath [0x77897B6E+286]
	RtlGetAppContainerNamedObjectPath [0x77897B3E+238]


In [ ]:
#listing_64624854 > div > div > div > div > div.fas4ee0 > a > img
#listing_64624839 > div:nth-child(1) > div > div > div > div.fas4ee0 > a > img

In [ ]:
<a aria-live="polite" class="x8jo560 x8jo563 x8jo56a _1ftx2fq8" 
data-loading="false" aria-disabled="true"><div class="_1ljm00u7 
_1ljm00u1n _1ljm00ur    "><div>Next</div><div class="x8jo56h"><svg class="_1n3szym1 _1n3szym8">
<use href="#arrow-right-medium"></use></svg></div></div></a>


<a aria-live="polite" class="x8jo560 x8jo563 x8jo56a _1ftx2fq8"
data-loading="false" aria-disabled="false" href="#__next"><div class="_1ljm00u7
_1ljm00u1n _1ljm00ur    "><div>Next</div><div class="x8jo56h"><svg class="_1n3szym1 _1n3szym8">
<use href="#arrow-right-medium"></use></svg></div></div></a>